<h1>Who is that Pokemon?!</h1>

In [44]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Parameters
TARGET_SIZE = (128, 128)
HOG_ORIENTATIONS = 9
HOG_PIXELS_PER_CELL = (8, 8)
HOG_CELLS_PER_BLOCK = (2, 2)
COLOR_BINS = 32

<h2>1. Set Up and Load the Data</h2>

In [45]:
# Load data for binary classification (Pokémon vs. No-Pokémon)
def load_binary_data(pokemon_folder, no_pokemon_folder):
    images, labels = [], []
    # Load Pokémon images (label 1)
    for class_name in os.listdir(pokemon_folder):
        class_path = os.path.join(pokemon_folder, class_name)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                img_path = os.path.join(class_path, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, TARGET_SIZE)
                    images.append(img)
                    labels.append(1)
    # Load No-Pokémon images (label 0)
    for img_file in os.listdir(no_pokemon_folder):
        img_path = os.path.join(no_pokemon_folder, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, TARGET_SIZE)
            images.append(img)
            labels.append(0)
    return images, np.array(labels)

In [46]:
# Load data for multi-class classification
def load_multiclass_data(pokemon_folder):
    images, labels = [], []
    for class_name in os.listdir(pokemon_folder):
        class_path = os.path.join(pokemon_folder, class_name)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                img_path = os.path.join(class_path, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, TARGET_SIZE)
                    images.append(img)
                    labels.append(class_name)
    return images, np.array(labels)

<h2>2. Preprocessing</h2>

In [47]:
from skimage.feature import local_binary_pattern
from sklearn.cluster import MiniBatchKMeans

def extract_features(images):
    features_list = []
    for img in images:
        # Original HOG Features
        hog_feat = hog(img, orientations=HOG_ORIENTATIONS,
                      pixels_per_cell=HOG_PIXELS_PER_CELL,
                      cells_per_block=HOG_CELLS_PER_BLOCK,
                      channel_axis=-1)
        
        # LBP Features (Texture)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=256, range=(0, 256))
        
        # Improved Color Features using HSV
        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        hist_h = np.histogram(hsv[:,:,0], bins=COLOR_BINS, range=(0,180))[0]
        hist_s = np.histogram(hsv[:,:,1], bins=COLOR_BINS, range=(0,256))[0]
        hist_v = np.histogram(hsv[:,:,2], bins=COLOR_BINS, range=(0,256))[0]
        
        # Combine all features
        combined_feat = np.concatenate([
            hog_feat,
            lbp_hist,
            hist_h,
            hist_s,
            hist_v
        ])
        features_list.append(combined_feat)
    
    return np.array(features_list)

def create_sift_features(images, n_clusters=50):
    sift = cv2.SIFT_create()
    descriptors = []
    
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        _, des = sift.detectAndCompute(gray, None)
        if des is not None:
            descriptors.append(des)
    
    # Flatten descriptors and cluster
    all_descriptors = np.vstack(descriptors)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(all_descriptors)
    
    # Create histograms
    sift_features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        _, des = sift.detectAndCompute(gray, None)
        if des is None:
            hist = np.zeros(n_clusters)
        else:
            clusters = kmeans.predict(des)
            hist = np.bincount(clusters, minlength=n_clusters)
        sift_features.append(hist)
    
    return np.array(sift_features)

In [48]:
from sklearn.preprocessing import StandardScaler

# Train Binary Classifier
pokemon_folder = 'pokemon-data'
no_pokemon_folder = 'no-pokemon'
binary_images, binary_labels = load_binary_data(pokemon_folder, no_pokemon_folder)

# Combine SIFT with other features (optional)
sift_features = create_sift_features(binary_images)
combined_features = np.hstack([extract_features(binary_images), sift_features])

# For binary classifier
scaler_bin = StandardScaler()
X_binary = scaler_bin.fit_transform(combined_features)
y_binary = binary_labels

X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(
    X_binary, y_binary, test_size=0.2, stratify=y_binary, random_state=42)

binary_clf = RandomForestClassifier(n_estimators=100, random_state=42)
binary_clf.fit(X_train_bin, y_train_bin)

# Evaluate Binary Classifier
y_pred_bin = binary_clf.predict(X_test_bin)
print("Binary Classifier Performance:")
print(classification_report(y_test_bin, y_pred_bin))

c:\Users\i.kasionis\AppData\Local\anaconda3\envs\msc\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2560 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


Binary Classifier Performance:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       580
           1       0.99      0.95      0.97       319

    accuracy                           0.98       899
   macro avg       0.98      0.97      0.98       899
weighted avg       0.98      0.98      0.98       899



In [49]:
from sklearn.preprocessing import LabelEncoder

# Train Multi-class Classifier (WITH LABEL ENCODING)
multi_images, multi_labels = load_multiclass_data(pokemon_folder)

# Encode string labels to integers
label_encoder = LabelEncoder()
y_multi_encoded = label_encoder.fit_transform(multi_labels)

# Combine SIFT with other features (optional)
sift_features = create_sift_features(multi_images)
combined_features = np.hstack([extract_features(multi_images), sift_features])

# For multi-class classifier
scaler_multi = StandardScaler()
X_multi = scaler_multi.fit_transform(combined_features)

X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X_multi, y_multi_encoded, test_size=0.2, stratify=y_multi_encoded, random_state=42
)

multi_clf = XGBClassifier(objective='multi:softmax', random_state=42)
multi_clf.fit(X_train_multi, y_train_multi)

# Evaluate Multi-class Classifier
y_pred_multi_encoded = multi_clf.predict(X_test_multi)
y_pred_multi = label_encoder.inverse_transform(y_pred_multi_encoded)  # Convert back to names

print("Multi-class Classifier Performance:")
print(classification_report(label_encoder.inverse_transform(y_test_multi), y_pred_multi))

c:\Users\i.kasionis\AppData\Local\anaconda3\envs\msc\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2560 or by setting the environment variable OMP_NUM_THREADS=4
  warnings.warn(


Multi-class Classifier Performance:
              precision    recall  f1-score   support

   Blastoise       0.72      0.72      0.72        29
   Bulbasaur       0.87      0.96      0.92        28
   Charizard       0.67      0.50      0.57        20
  Charmander       0.74      0.71      0.72        24
  Charmeleon       0.96      0.90      0.93        29
     Ivysaur       0.74      0.87      0.80        30
     Pikachu       0.92      0.85      0.88        39
      Raichu       0.67      0.85      0.75        33
    Squirtle       0.95      0.65      0.77        31
    Venusaur       0.79      0.68      0.73        28
   Wartortle       0.67      0.86      0.75        28

    accuracy                           0.79       319
   macro avg       0.79      0.78      0.78       319
weighted avg       0.80      0.79      0.79       319



In [56]:
def segment_image(image, min_area=500):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Improved preprocessing
    blurred = cv2.bilateralFilter(gray, 9, 75, 75)
    thresh = cv2.adaptiveThreshold(blurred, 255, 
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY_INV, 11, 2)
    
    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=3)
    
    # Edge-aware refinement
    edges = cv2.Canny(closed, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    rois = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > min_area:
            x, y, w, h = cv2.boundingRect(cnt)
            # Add padding to capture full shapes
            pad = 5
            rois.append(image[max(0,y-pad):min(y+h+pad, image.shape[0]),
                             max(0,x-pad):min(x+w+pad, image.shape[1])])
    return rois

# Updated Inference Pipeline
def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return []
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Binary classification on full image
    img_resized = cv2.resize(img_rgb, TARGET_SIZE)
    features = extract_features([img_resized])[0]
    sift_features = create_sift_features([img_resized])[0]
    combined_features = np.hstack([features, sift_features])
    if binary_clf.predict([combined_features])[0] == 0:
        return []
    
    # Segmentation and classification
    rois = segment_image(img_rgb)
    predictions = []
    for roi in rois:
        roi_resized = cv2.resize(roi, TARGET_SIZE)
        roi_features = extract_features([roi_resized])[0]
        sift_features = create_sift_features([roi_resized])[0]
        combined_features = np.hstack([roi_features, sift_features])
        if binary_clf.predict([combined_features])[0] == 1:
            pred_encoded = multi_clf.predict([combined_features])[0]
            pred = label_encoder.inverse_transform([pred_encoded])[0]  # Decode label
            predictions.append(pred)
    return predictions

In [57]:
# Example usage
test_image_path = 'test_image.png'
print("Predictions:", predict_image(test_image_path))

c:\Users\i.kasionis\AppData\Local\anaconda3\envs\msc\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2560 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


ValueError: n_samples=19 should be >= n_clusters=50.